<a href="https://colab.research.google.com/github/taerri/meal-kit-analysis/blob/main/%EB%B9%85%EC%B9%B4%EC%9D%B8%EC%A6%88_%EB%B0%80%ED%82%A4%ED%8A%B8_TF-IDF_LDA_SNA_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 밀키트 보완관계 분석
- 빅카인즈
---

- 데이터 읽기

In [ ]:
import pandas as pd

In [ ]:
Corpus_all = pd.read_csv("mealkit_preprocessing.csv", encoding="utf8")
print("데이터 개수 :",len(Corpus_all))
Corpus_all.head()

데이터 개수 : 5529


,Unnamed: 0,키워드
0,0,"재배,채소,제공,웰빙,먹거리,하이원리조트,요리사,셰프들,재배,채소,식재료,활용,손님..."
1,1,"백화점,롯데쇼핑,매출,역성장,실적,롯데쇼핑,백화점,회복세,롯데마트,롯데하이마트,e커..."
2,2,"밀키트,창업,가정,안산시,밀키트,기탁,브랜드,100호,밀키트,창업,브랜드,최단,기간..."
3,3,"수산물,할인,정부,소비,촉진,정부,할인,쿠폰,발행,지원,추가,코로나19,최소화,전통..."
4,4,"소상공인,구독,경제,바우처,제공,소상공인,구독경제,중소벤처기업부,비상,경제,중앙,대..."


# 단어 빈도

In [ ]:
Corpus_org = Corpus_all["키워드"]
#Words_2019 = Corpus_all[Corpus_all.Date < "2020/01/01"]["all_text"]
#Words_2020 = Corpus_all[Corpus_all.Date >= "2020/01/01"]["all_text"]
Corpus = [wl.split(",")  for wl in Corpus_org]

In [ ]:
def isHangulOrAlphabet(word):
    return word[0].isalpha() 


In [ ]:
isHangulOrAlphabet("12")

False

In [ ]:
for wl in Corpus[:3]:
  print(wl)

['재배', '채소', '제공', '웰빙', '먹거리', '하이원리조트', '요리사', '셰프들', '재배', '채소', '식재료', '활용', '손님', '웰빙', '먹거리', '제공', '호평', '셰프들', '하이원리조트', '호텔', '셰프', '주인공', '호텔', '셰프', '키친', '오가닉팜', '운영', '고객', '건강', '먹거리', '새싹', '허브', '제공', '고객서비스', '에코', '캠페인', '키친가든', '식재료', '활용', '채소', '허브', '열매', '정원', '키친', '오가닉팜', '하이원리조트', '운암정', '조성', '호텔', '셰프', '구슬땀', '20여종', '채소', '허브', '재배', '재배', '채소', '허브', '호텔', '위치', '식음업장', '식재료', '활용', '하이원리조트', 'PB', '상품', 'GOURMET', '고메', '밀키트', '제품', '이용', '채소', '허브', '농법', '재배', '강원랜드', '사내', '벤처', '자원업사이클TF', '도움', '리조트', '배출', '음식물', '쓰레기', '동애등에', '곤충', '먹이', '활용', '자연', '분해', '퇴비', '지원', '활용', '하이원리조트', '조리', '이태규', '부장', '하이원리조트', '호텔', '손님', '정성', '재배', '식재료', '활용', '음식', '대접', '수요', '음식', '노력']
['백화점', '롯데쇼핑', '매출', '역성장', '실적', '롯데쇼핑', '백화점', '회복세', '롯데마트', '롯데하이마트', 'e커머스', '롯데온', '주력', '계열사', '부진', '매출', '역성장', '구조', '조정', '성과', '영업이익', '444%', '증가', '시장', '롯데쇼핑', '영업', '이익', '75억', '444.7%', '동기', '대비', '444.7%', '공시', '매출액', '3.5%', '감소', '영업이익', '송도롯데몰', '공사

In [ ]:
def Word_count(corpus):
    word_cnt = dict()
    for text in corpus:
        for word in text:
            if word_cnt.get(word):
                word_cnt[word]+=1
            else:
                word_cnt[word]=1
    cnt_df = pd.DataFrame.from_dict(word_cnt, orient='index')
    cnt_df.columns = ["개수"]
    cnt_df = cnt_df.sort_values(["개수"],ascending=False)
    return word_cnt, cnt_df

In [ ]:
Words_cnt, Cnt_df = Word_count(Corpus)
#Words_2019_cnt = Word_count(Words_2019)
#Words_2020_cnt = Word_count(Words_2020)

In [ ]:
# 저장
Cnt_df.to_csv("단어빈도.csv", encoding="utf-8-sig")
#Words_2019_cnt.to_csv("./단어빈도/단어빈도_2019.csv",encoding="cp949")
#Words_2020_cnt.to_csv("./단어빈도/단어빈도_2020.csv",encoding="cp949")

In [ ]:
def is_good_word(word):
    if len(word) == 0:
        return False
    # 숫자로 시작하는 단어 제외
    if word[0].isdigit():
        return False
    # 빈도수 n 이하이면 제외
    if Words_cnt.get(word, 0) < 5: 
        return False
    return True

In [ ]:
Good_Corpus = []
for wl in Corpus:
    wl2 = [w for w in wl if is_good_word(w)]
    Good_Corpus.append(wl2)

In [ ]:
for wl in Good_Corpus[:3]:
    print(wl)

['재배', '채소', '제공', '웰빙', '먹거리', '하이원리조트', '요리사', '셰프들', '재배', '채소', '식재료', '활용', '손님', '웰빙', '먹거리', '제공', '호평', '셰프들', '하이원리조트', '호텔', '셰프', '주인공', '호텔', '셰프', '키친', '오가닉팜', '운영', '고객', '건강', '먹거리', '허브', '제공', '고객서비스', '에코', '캠페인', '식재료', '활용', '채소', '허브', '열매', '정원', '키친', '오가닉팜', '하이원리조트', '조성', '호텔', '셰프', '구슬땀', '채소', '허브', '재배', '재배', '채소', '허브', '호텔', '위치', '식재료', '활용', '하이원리조트', 'PB', '상품', '고메', '밀키트', '제품', '이용', '채소', '허브', '재배', '사내', '벤처', '도움', '리조트', '배출', '음식물', '쓰레기', '곤충', '먹이', '활용', '자연', '분해', '퇴비', '지원', '활용', '하이원리조트', '조리', '부장', '하이원리조트', '호텔', '손님', '정성', '재배', '식재료', '활용', '음식', '대접', '수요', '음식', '노력']
['백화점', '롯데쇼핑', '매출', '역성장', '실적', '롯데쇼핑', '백화점', '회복세', '롯데마트', '롯데하이마트', 'e커머스', '롯데온', '주력', '계열사', '부진', '매출', '역성장', '구조', '조정', '성과', '영업이익', '증가', '시장', '롯데쇼핑', '영업', '이익', '동기', '대비', '공시', '매출액', '감소', '영업이익', '공사', '지연', '세금', '영향', '비용', '제외', '영업', '시장', '롯데쇼핑', '매출', '영업이익', '기록', '예상', '보복', '소비', '효과', '백화점', '매출', '영업이익', '성장', '백화점', '매출', '영업이

---
# TF-IDF
사이킷런은 TF-IDF를 자동 계산해주는 TfidfVectorizer를 제공

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def TF_IDF(corpus):
    docs = [" ".join(x) for x in list(corpus)]
    print("--", docs[0])
    print("--", docs[1])
    tfidf_vector = TfidfVectorizer().fit(docs)
    tfidf_array = tfidf_vector.transform(docs).toarray()
    #print(tf_idf)
    return tfidf_vector, tfidf_array


In [ ]:
tfidf_vector, tfidf_array = TF_IDF(Good_Corpus)


-- 재배 채소 제공 웰빙 먹거리 하이원리조트 요리사 셰프들 재배 채소 식재료 활용 손님 웰빙 먹거리 제공 호평 셰프들 하이원리조트 호텔 셰프 주인공 호텔 셰프 키친 오가닉팜 운영 고객 건강 먹거리 허브 제공 고객서비스 에코 캠페인 식재료 활용 채소 허브 열매 정원 키친 오가닉팜 하이원리조트 조성 호텔 셰프 구슬땀 채소 허브 재배 재배 채소 허브 호텔 위치 식재료 활용 하이원리조트 PB 상품 고메 밀키트 제품 이용 채소 허브 재배 사내 벤처 도움 리조트 배출 음식물 쓰레기 곤충 먹이 활용 자연 분해 퇴비 지원 활용 하이원리조트 조리 부장 하이원리조트 호텔 손님 정성 재배 식재료 활용 음식 대접 수요 음식 노력
-- 백화점 롯데쇼핑 매출 역성장 실적 롯데쇼핑 백화점 회복세 롯데마트 롯데하이마트 e커머스 롯데온 주력 계열사 부진 매출 역성장 구조 조정 성과 영업이익 증가 시장 롯데쇼핑 영업 이익 동기 대비 공시 매출액 감소 영업이익 공사 지연 세금 영향 비용 제외 영업 시장 롯데쇼핑 매출 영업이익 기록 예상 보복 소비 효과 백화점 매출 영업이익 성장 백화점 매출 영업이익 급증 회복 소비 지속 해외 사업 기조 효과 영향 고성장 설명 신장률 기존점 신장 만큼 회복 성공 모습 할인 롯데마트 지점 축소 매출 감소 부진 점포 축소 매출 구조 조정 효과 개선 주류 신선식품 밀키트 자릿수 성장 기존점 신장 성장 롯데하이마트 여름 기온 에어컨 판매 부진 매출 동기 대비 매출액 영업이익 성장세 제동 온라인 성장 동기 대비 증가 무더위 시작 에어컨 판매 실적 반등 예상 슈퍼 매출 감소 부진 홈쇼핑 마케팅 강화 영업이익 매출액 기저 효과 영향 매출액 기록 e커머스 부진 사업부 매출 기록 셀러 수수료 인하 영향 영향 회계 기준 변경 반영 설명 영업 적자 적자 확대 광고 판관비 안정 확대 비용 증가 롯데쇼핑 관계자 오픈 롯데온 회계 기준 영향 반영 거래액 트래픽 지속적 증가


In [ ]:
tfidf_sk = pd.DataFrame(tfidf_array, columns = tfidf_vector.get_feature_names())
tfidf_sk

,aaf,abc,ac,active,ageless,ai,aia,aia생명,ai장,ajunews,...,히든,히어로,히트,히트상품,히트작,힌트,힐링,힘줄,힙플레이스,힙합
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5524,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5525,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5526,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
vocab_list = tfidf_vector.get_feature_names()

In [ ]:
with open("mealkit_words.txt", "w") as fp:
    for v in vocab_list:
        fp.write("%s\n" % v)
        
print("saved to mealkit_words.txt")

saved to mealkit_words.txt


### tf_idf 결과 csv 저장하기 

In [ ]:
tfidf_sk.to_csv("mealkit_tfidf1.csv", encoding="utf-8-sig")

In [ ]:
print(len(tfidf_vector.vocabulary_))

15369


In [ ]:
tfidf_vector.vocabulary_

{'재배': 10795,
 '채소': 12273,
 '제공': 11262,
 '웰빙': 9664,
 '먹거리': 4413,
 '하이원리조트': 14337,
 '요리사': 9382,
 '셰프들': 6991,
 '식재료': 7846,
 '활용': 15176,
 '손님': 7154,
 '호평': 14934,
 '호텔': 14923,
 '셰프': 6990,
 '주인공': 11619,
 '키친': 13222,
 '오가닉팜': 9095,
 '운영': 9531,
 '고객': 1383,
 '건강': 1113,
 '허브': 14731,
 '고객서비스': 1389,
 '에코': 8746,
 '캠페인': 12906,
 '열매': 8948,
 '정원': 11208,
 '조성': 11433,
 '구슬땀': 1906,
 '위치': 9715,
 'pb': 476,
 '상품': 6450,
 '고메': 1439,
 '밀키트': 4952,
 '제품': 11354,
 '이용': 10160,
 '사내': 6097,
 '벤처': 5463,
 '도움': 3496,
 '리조트': 4116,
 '배출': 5327,
 '음식물': 9956,
 '쓰레기': 8168,
 '곤충': 1530,
 '먹이': 4419,
 '자연': 10595,
 '분해': 5837,
 '퇴비': 13464,
 '지원': 11902,
 '조리': 11388,
 '부장': 5749,
 '정성': 11194,
 '음식': 9948,
 '대접': 3302,
 '수요': 7316,
 '노력': 2804,
 '백화점': 5367,
 '롯데쇼핑': 4037,
 '매출': 4370,
 '역성장': 8843,
 '실적': 8085,
 '회복세': 15206,
 '롯데마트': 4030,
 '롯데하이마트': 4057,
 'e커머스': 192,
 '롯데온': 4042,
 '주력': 11560,
 '계열사': 1368,
 '부진': 5757,
 '구조': 1919,
 '조정': 11458,
 '성과': 6826,
 '영업이익': 9017,
 '증가':

In [ ]:
print(type(tfidf_array))
print(tfidf_array.shape)

<class 'numpy.ndarray'>
(5529, 15369)


# LDA

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
import pickle
from gensim import corpora
import gensim


C:\Users\korea\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [ ]:
class LDA_model:
    def __init__(self,corpus):
        self.corpus = corpus
        return;
    def mk_detokenized_doc(self):
        self.detokenized_doc = []
        for i in range(len(self.corpus)):
            try:
                t = ' '.join(self.corpus.iloc[i])
                self.detokenized_doc.append(t)
            except:
                print(i)
        return;
    def tf_idf_vector(self,max_features):
        self.vectorizer = TfidfVectorizer(max_features= max_features) # 상위 1000개의 단어 보존
        self.X = self.vectorizer.fit_transform(self.detokenized_doc)
        return;
    def mk_word2id(self):
        self.word2id = defaultdict(lambda : 0)
        for idx, feature in enumerate(self.vectorizer.get_feature_names()):
            self.word2id[feature] = idx
        return;
    def print_word2id(self,n=1):
        for i, sent in enumerate(self.detokenized_doc):
            print('====== document[%d] ======' % i)
            print( [ (token, self.X[i, self.word2id[token]]) for token in sent.split() ] )
            if i==n:
                break
        return;
    def mk_lda_model(self,n_topic=10,max_iter=1):
        self.lda_model=LatentDirichletAllocation(n_components=n_topic,learning_method='online',random_state=777,max_iter=max_iter)
        self.lda_top=self.lda_model.fit_transform(self.X)
        return;
    def print_lda(self):
        print("-------------------------------------")
        print(self.lda_model.components_)
        print(self.lda_model.components_.shape) 
        return;
    def print_topic_lda(self,n=5):
        print("-------------------------------------")
        terms = self.vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.
        for idx, topic in enumerate(self.lda_model.components_):
            print("Topic %d:" % (idx+1), [(terms[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
        return;

In [ ]:
model_all = LDA_model(Corpus)

# LDA 시각화

- 토픽별 단어 분포

In [ ]:
dictionary = corpora.Dictionary(model_all.corpus)
corpus = [dictionary.doc2bow(text) for text in model_all.corpus]
NUM_TOPICS = 3
num_words = 30
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=num_words)
for topic in topics:
    print(topic)
dictionary.save('dictionary_전처리.gensim')
pickle.dump(corpus, open('corpus_전처리.pkl', 'wb'))
ldamodel.save('gensim_model_전처리.gensim')



(0, '0.025*"밀키트" + 0.015*"제품" + 0.014*"시장" + 0.010*"요리" + 0.008*"식품" + 0.008*"출시" + 0.007*"메뉴" + 0.007*"간편식" + 0.007*"배송" + 0.006*"브랜드" + 0.006*"판매" + 0.005*"서비스" + 0.005*"상품" + 0.005*"조리" + 0.005*"확대" + 0.004*"성장" + 0.004*"소비자" + 0.004*"한국야쿠르트" + 0.004*"식재료" + 0.004*"매출" + 0.004*"음식" + 0.004*"프레시지" + 0.004*"셰프" + 0.004*"배달" + 0.004*"재료" + 0.003*"고객" + 0.003*"대표" + 0.003*"사업" + 0.003*"인기" + 0.003*"레시피"')
(1, '0.017*"상품" + 0.012*"판매" + 0.010*"매출" + 0.009*"밀키트" + 0.009*"고객" + 0.008*"서비스" + 0.008*"증가" + 0.007*"구매" + 0.007*"할인" + 0.006*"온라인" + 0.006*"배송" + 0.006*"세트" + 0.006*"선물" + 0.006*"이마트" + 0.006*"코로나19" + 0.005*"대비" + 0.005*"인기" + 0.005*"주문" + 0.004*"행사" + 0.004*"진행" + 0.004*"기간" + 0.004*"가격" + 0.004*"매장" + 0.004*"제공" + 0.003*"최대" + 0.003*"명절" + 0.003*"준비" + 0.003*"소비" + 0.003*"이용" + 0.003*"제품"')
(2, '0.008*"지원" + 0.006*"사업" + 0.006*"밀키트" + 0.006*"기업" + 0.005*"코로나19" + 0.005*"지역" + 0.004*"대표" + 0.004*"온라인" + 0.004*"진행" + 0.003*"시장" + 0.003*"소상공인" + 0.003*"선정" + 0.003*"제공" + 0.003*"서비

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.save_html(vis,"lda_전처리.html")

C:\Users\korea\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [ ]:
len(dictionary) # 각 단어의 정수형 표시

63967

In [ ]:
NUM_TOPICS = 3
num_words = 30
#ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=num_words)
for topic in topics:
    print(topic)

(0, '0.025*"밀키트" + 0.015*"제품" + 0.014*"시장" + 0.010*"요리" + 0.008*"식품" + 0.008*"출시" + 0.007*"메뉴" + 0.007*"간편식" + 0.007*"배송" + 0.006*"브랜드" + 0.006*"판매" + 0.005*"서비스" + 0.005*"상품" + 0.005*"조리" + 0.005*"확대" + 0.004*"성장" + 0.004*"소비자" + 0.004*"한국야쿠르트" + 0.004*"식재료" + 0.004*"매출" + 0.004*"음식" + 0.004*"프레시지" + 0.004*"셰프" + 0.004*"배달" + 0.004*"재료" + 0.003*"고객" + 0.003*"대표" + 0.003*"사업" + 0.003*"인기" + 0.003*"레시피"')
(1, '0.017*"상품" + 0.012*"판매" + 0.010*"매출" + 0.009*"밀키트" + 0.009*"고객" + 0.008*"서비스" + 0.008*"증가" + 0.007*"구매" + 0.007*"할인" + 0.006*"온라인" + 0.006*"배송" + 0.006*"세트" + 0.006*"선물" + 0.006*"이마트" + 0.006*"코로나19" + 0.005*"대비" + 0.005*"인기" + 0.005*"주문" + 0.004*"행사" + 0.004*"진행" + 0.004*"기간" + 0.004*"가격" + 0.004*"매장" + 0.004*"제공" + 0.003*"최대" + 0.003*"명절" + 0.003*"준비" + 0.003*"소비" + 0.003*"이용" + 0.003*"제품"')
(2, '0.008*"지원" + 0.006*"사업" + 0.006*"밀키트" + 0.006*"기업" + 0.005*"코로나19" + 0.005*"지역" + 0.004*"대표" + 0.004*"온라인" + 0.004*"진행" + 0.003*"시장" + 0.003*"소상공인" + 0.003*"선정" + 0.003*"제공" + 0.003*"서비

---
# 불러와서 분석

In [ ]:
dictionary = gensim.corpora.Dictionary.load('dictionary_전처리.gensim')
corpus = pickle.load(open('corpus_전처리.pkl', 'rb'))
ldamodel = gensim.models.ldamodel.LdaModel.load('gensim_model_전처리.gensim')

# 네트워크 가즈아_